In [9]:
# Import các thư viện cần thiết
import xml.etree.ElementTree as ET
import pandas as pd
import os

# Đường dẫn đến file XML
xml_path = "Dataset/VSoSLCSum.xml"

# Parse file XML
tree = ET.parse(xml_path)
root = tree.getroot()

print(f"Số lượng posts: {len(root.findall('post'))}")

Số lượng posts: 141


In [10]:
# Hàm trích xuất dữ liệu từ XML
def extract_data_from_xml(root):
    """
    Trích xuất dữ liệu từ XML với cấu trúc:
    - title: Tiêu đề bài viết
    - content_docs: Tất cả các câu từ document (đánh số)
    - content_comment: Tất cả các câu từ comments (đánh số)
    - golden_docs: Các câu document được chọn vào summary
    - golden_comment: Các câu comment được chọn vào summary
    """
    data = []
    
    for post in root.findall('post'):
        post_id = post.get('id')
        
        # Lấy title
        title_elem = post.find('title')
        title = title_elem.text.strip() if title_elem is not None and title_elem.text else ""
        
        # Lấy summary (golden summary) - giữ lại để tham khảo
        summary_sentences = []
        summary_elem = post.find('summary/sentences')
        if summary_elem is not None:
            for sentence in summary_elem.findall('sentence'):
                content_elem = sentence.find('content')
                if content_elem is not None and content_elem.text:
                    summary_sentences.append(content_elem.text.strip())
        golden_full = " ".join(summary_sentences)
        
        # Lấy document sentences
        doc_sentences = []
        doc_elem = post.find('document/sentences')
        if doc_elem is not None:
            for idx, sentence in enumerate(doc_elem.findall('sentence')):
                content_elem = sentence.find('content')
                in_summary = sentence.get('in_summary', '0')
                if content_elem is not None and content_elem.text:
                    doc_sentences.append({
                        'index': idx,
                        'content': content_elem.text.strip(),
                        'in_summary': int(in_summary) if in_summary.isdigit() else 0
                    })
        
        # Lấy comment sentences
        comment_sentences = []
        comments_elem = post.find('comments/comment/sentences')
        if comments_elem is not None:
            for idx, sentence in enumerate(comments_elem.findall('sentence')):
                content_elem = sentence.find('content')
                in_summary = sentence.get('in_summary', '0')
                if content_elem is not None and content_elem.text:
                    comment_sentences.append({
                        'index': idx,
                        'content': content_elem.text.strip(),
                        'in_summary': int(in_summary) if in_summary.isdigit() else 0
                    })
        
        # Tạo content_docs string (đánh số các câu document)
        content_docs_list = [f"{i}: {sent['content']}" for i, sent in enumerate(doc_sentences)]
        content_docs = "\n".join(content_docs_list)
        
        # Tạo content_comment string (đánh số các câu comment)
        content_comment_list = [f"{i}: {sent['content']}" for i, sent in enumerate(comment_sentences)]
        content_comment = "\n".join(content_comment_list)
        
        # Lấy golden_docs: các câu document được chọn vào summary (in_summary >= 1)
        golden_docs_sentences = [sent['content'] for sent in doc_sentences if sent['in_summary'] >= 1]
        golden_docs = " ".join(golden_docs_sentences)
        
        # Lấy golden_comment: các câu comment được chọn vào summary (in_summary >= 1)
        golden_comment_sentences = [sent['content'] for sent in comment_sentences if sent['in_summary'] >= 1]
        golden_comment = " ".join(golden_comment_sentences)
        
        # Lấy các index của câu được chọn
        selected_docs_indices = [sent['index'] for sent in doc_sentences if sent['in_summary'] >= 1]
        selected_comment_indices = [sent['index'] for sent in comment_sentences if sent['in_summary'] >= 1]
        
        data.append({
            'post_id': post_id,
            'title': title,
            'content_docs': content_docs,
            'content_comment': content_comment,
            'golden_docs': golden_docs,
            'golden_comment': golden_comment,
            'golden_full': golden_full,
            'selected_docs_indices': selected_docs_indices,
            'selected_comment_indices': selected_comment_indices,
            'num_docs': len(doc_sentences),
            'num_comments': len(comment_sentences)
        })
    
    return data

# Trích xuất dữ liệu
data = extract_data_from_xml(root)
print(f"Đã trích xuất {len(data)} posts")

# Xem mẫu dữ liệu đầu tiên
if data:
    print("\n--- Mẫu dữ liệu đầu tiên ---")
    print(f"Post ID: {data[0]['post_id']}")
    print(f"Title: {data[0]['title']}")
    print(f"Số câu docs: {data[0]['num_docs']}")
    print(f"Số câu comments: {data[0]['num_comments']}")
    print(f"Selected docs indices: {data[0]['selected_docs_indices']}")
    print(f"Selected comment indices: {data[0]['selected_comment_indices']}")

Đã trích xuất 141 posts

--- Mẫu dữ liệu đầu tiên ---
Post ID: yen-bai_vne_2
Title: Tình_cảm của hàng_xóm với Bí_thư Tỉnh_ủy Yên_Bái
Số câu docs: 26
Số câu comments: 29
Selected docs indices: [0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 20, 24]
Selected comment indices: [0, 1, 6, 7, 9, 11, 13, 14, 15, 17, 21, 23, 24, 25]


In [11]:
# Tạo DataFrame và lưu CSV
df = pd.DataFrame(data)

# Hiển thị thông tin DataFrame
print("Thông tin DataFrame:")
print(df.info())
print(f"\nSố dòng: {len(df)}")
print(f"\nCác cột: {df.columns.tolist()}")

# Kiểm tra xem có đủ cột mới không
required_cols = ['content_docs', 'content_comment', 'golden_docs', 'golden_comment']
missing_cols = [col for col in required_cols if col not in df.columns]
if missing_cols:
    print(f"\n⚠️ CẢNH BÁO: Thiếu các cột: {missing_cols}")
    print("Vui lòng chạy lại Cell 2 (hàm extract_data_from_xml) trước!")
else:
    print("\n✅ Đã có đủ các cột mới!")

# Xem vài dòng đầu
df.head()

Thông tin DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141 entries, 0 to 140
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   post_id                   141 non-null    object
 1   title                     141 non-null    object
 2   content_docs              141 non-null    object
 3   content_comment           141 non-null    object
 4   golden_docs               141 non-null    object
 5   golden_comment            141 non-null    object
 6   golden_full               141 non-null    object
 7   selected_docs_indices     141 non-null    object
 8   selected_comment_indices  141 non-null    object
 9   num_docs                  141 non-null    int64 
 10  num_comments              141 non-null    int64 
dtypes: int64(2), object(9)
memory usage: 12.2+ KB
None

Số dòng: 141

Các cột: ['post_id', 'title', 'content_docs', 'content_comment', 'golden_docs', 'golden_comment', 'go

,post_id,title,content_docs,content_comment,golden_docs,golden_comment,golden_full,selected_docs_indices,selected_comment_indices,num_docs,num_comments
0,yen-bai_vne_2,Tình_cảm của hàng_xóm với Bí_thư Tỉnh_ủy Yên_Bái,"0: Sáng 19/8 , thành_phố Yên_Bái mưa không ngớ...","0: Tại_sao như_vậy ? Phải có nguyên_nhân , Chú...","Sáng 19/8 , thành_phố Yên_Bái mưa không ngớt d...","Tại_sao như_vậy ? Phải có nguyên_nhân , Chúng_...",Bí_thư Tỉnh_uỷ Yên_Bái_Phạm_Duy_Cường được hàn...,"[0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[0, 1, 6, 7, 9, 11, 13, 14, 15, 17, 21, 23, 24...",26,29
1,tan-son-nhat_zing_7,Lãnh_đạo sân_bay Tân_Sơn_Nhất phủ_nhận chuyện ...,"0: Chiều 26/8 , mưa to kéo_dài khiến nhiều nơi...",0: Bến du_thuyền TÂN_SƠN_NHẤT mới khai_trương ...,"Chiều 26/8 , mưa to kéo_dài khiến nhiều nơi ở ...",Bến du_thuyền TÂN_SƠN_NHẤT mới khai_trương . ....,Cơn mưa lớn chiều 26/8 làm sân_bay Tân_Sơn_Nhấ...,"[0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[0, 1, 6, 10, 11, 12, 16, 21, 23, 24, 25, 26, ...",16,191
2,donald-trump_vne_52,Donald_Trump nói Tổng_thống Obama là ' người s...,0: Ứng_viên tổng_thống đảng Cộng_hòa Donald_Tr...,"0: giờ thì tôi không còn tin ông ấy sẽ thắng ,...",Ứng_viên tổng_thống đảng Cộng_hòa Donald_Trump...,"giờ thì tôi không còn tin ông ấy sẽ thắng , nh...",Donald_Trump cáo_buộc Tổng_thống Mỹ_Barack_Oba...,"[0, 1, 2, 3, 4, 5, 6, 8, 9]","[0, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 15, 1...",11,37
3,michael-phelps_zing_6,"Giành HCV thứ 21 , Michael_Phelps hồi_sinh","0: Ở nội_dung sở_trường 200 m bướm , Phelps - ...",0: Qua 5 mùa olympic mà thành_tích vẫn sáng ch...,"Ở nội_dung sở_trường 200 m bướm , Phelps - đã ...",Qua 5 mùa olympic mà thành_tích vẫn sáng chói ...,Siêu kình_ngư Michael_Phelps tiếp_tục cho thấy...,"[0, 2, 3, 5, 6, 7, 8, 9, 10, 12, 16, 17, 18, 2...","[0, 1, 3, 4, 6, 7, 9, 10, 11, 12, 13, 14, 17, ...",24,120
4,casa_dantri_3,Danh_sách 9 cán_bộ chiến_sĩ trên máy_bay CASA-212,0: Lê_Kiêm_Toàn - Đại_tá Lữ_đoàn trưởng Lữ_đoà...,0: Hãy trở_về thôi các anh !\n1: cầu_mong các ...,Lê_Kiêm_Toàn - Đại_tá Lữ_đoàn trưởng Lữ_đoàn 9...,Hãy trở_về thôi các anh ! cầu_mong các anh trở...,Dân_trí Chiếc máy_bay CASA chở 9 người đã rơi ...,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]","[0, 1, 2, 3, 4, 5, 7, 10, 12, 13, 14, 15, 16]",11,17


In [12]:
# Lưu file CSV với các cột theo yêu cầu: title, content_docs, content_comment, golden_docs, golden_comment
# Tạo thư mục output nếu chưa có
output_dir = "Dataset/Done"
os.makedirs(output_dir, exist_ok=True)

# Lưu file CSV chính với format mới
df_main = df[['post_id', 'title', 'content_docs', 'content_comment', 'golden_docs', 'golden_comment']]
df_main.to_csv(f"{output_dir}/data_full.csv", index=False, encoding='utf-8-sig')
print(f"Đã lưu file: {output_dir}/data_full.csv")

# Lưu file CSV đơn giản theo format yêu cầu (title, content_docs, content_comment, golden_docs, golden_comment)
df_simple = df[['title', 'content_docs', 'content_comment', 'golden_docs', 'golden_comment']]
df_simple.to_csv(f"{output_dir}/data_summary.csv", index=False, encoding='utf-8-sig')
print(f"Đã lưu file: {output_dir}/data_summary.csv")

print(f"\nTổng số mẫu: {len(df_simple)}")
print(f"\nCác cột trong file CSV:")
print(df_simple.columns.tolist())

Đã lưu file: Dataset/Done/data_full.csv
Đã lưu file: Dataset/Done/data_summary.csv

Tổng số mẫu: 141

Các cột trong file CSV:
['title', 'content_docs', 'content_comment', 'golden_docs', 'golden_comment']


## Thử nghiệm với Gemini API

Phần này sử dụng Google Gemini API để:
1. Chọn câu phù hợp với chủ đề
2. Tóm tắt các câu bình luận

In [ ]:
# Cài đặt thư viện Google Generative AI (chạy 1 lần)
# !pip install google-generativeai

import google.generativeai as genai

# Cấu hình API Key - Thay YOUR_API_KEY bằng API key của bạn
# Lấy API key tại: https://aistudio.google.com/app/apikey
GEMINI_API_KEY = "YOUR_API_KEY"  # <-- Thay thế bằng API key của bạn

genai.configure(api_key=GEMINI_API_KEY)

# Khởi tạo model
model = genai.GenerativeModel('gemini-1.5-flash')

print("Đã cấu hình Gemini API thành công!")

In [ ]:
# Hàm gọi Gemini API để chọn câu
def select_sentences_with_gemini(title, content, num_sentences=6):
    """
    Sử dụng Gemini để chọn các câu phù hợp với chủ đề
    """
    prompt = f"""Bạn là một chuyên gia phân tích văn bản tiếng Việt. 
Hãy chọn ra {num_sentences} câu phù hợp nhất với chủ đề sau: {title}

<list câu>
{content}
</list câu>

Trả về CHÍNH XÁC dưới dạng danh sách số, ví dụ: [1, 2, 3, 5, 8, 10]
Chỉ trả về danh sách số, không giải thích."""

    try:
        response = model.generate_content(prompt)
        return response.text.strip()
    except Exception as e:
        return f"Lỗi: {str(e)}"

# Hàm gọi Gemini API để tóm tắt
def summarize_with_gemini(title, content):
    """
    Sử dụng Gemini để tóm tắt các câu bình luận
    """
    prompt = f"""Bạn là một chuyên gia tóm tắt văn bản tiếng Việt.
Hãy tóm tắt các bình luận dưới đây theo chủ đề: {title}

<list câu bình luận>
{content}
</list câu bình luận>

Yêu cầu:
- Tóm tắt ngắn gọn, súc tích
- Giữ lại các ý kiến quan trọng
- Viết bằng tiếng Việt tự nhiên

Output:"""

    try:
        response = model.generate_content(prompt)
        return response.text.strip()
    except Exception as e:
        return f"Lỗi: {str(e)}"

print("Đã định nghĩa các hàm gọi Gemini API")

In [ ]:
# Thử nghiệm với 1 mẫu dữ liệu
sample_idx = 0
sample = df.iloc[sample_idx]

print("=" * 60)
print("THỬ NGHIỆM VỚI GEMINI API")
print("=" * 60)

print(f"\n📌 CHỦ ĐỀ: {sample['title']}")
print(f"\n📝 SỐ CÂU BÌNH LUẬN: {sample['num_comments']}")

# 1. Chọn câu với Gemini
print("\n" + "-" * 60)
print("🔍 TASK 1: CHỌN CÂU PHÙ HỢP")
print("-" * 60)

gemini_selection = select_sentences_with_gemini(
    sample['title'], 
    sample['content'],
    num_sentences=6
)

print(f"\n✅ Gemini chọn: {gemini_selection}")
print(f"📊 Ground Truth: {sample['selected_indices']}")

In [ ]:
# 2. Tóm tắt với Gemini
print("\n" + "-" * 60)
print("📝 TASK 2: TÓM TẮT BÌNH LUẬN")
print("-" * 60)

gemini_summary = summarize_with_gemini(
    sample['title'], 
    sample['content']
)

print(f"\n✅ Gemini tóm tắt:")
print(gemini_summary)

print(f"\n📊 Ground Truth Summary:")
print(sample['golden'][:500] + "..." if len(sample['golden']) > 500 else sample['golden'])

In [ ]:
# Chạy batch trên nhiều mẫu (giới hạn để tránh rate limit)
import time

def process_batch_with_gemini(df, num_samples=5, delay=1):
    """
    Xử lý batch với Gemini API
    delay: thời gian chờ giữa các request (giây)
    """
    results = []
    
    for idx in range(min(num_samples, len(df))):
        sample = df.iloc[idx]
        print(f"\n🔄 Đang xử lý mẫu {idx + 1}/{num_samples}: {sample['post_id']}")
        
        # Chọn câu
        selection = select_sentences_with_gemini(
            sample['title'], 
            sample['content'],
            num_sentences=6
        )
        
        # Tóm tắt
        summary = summarize_with_gemini(
            sample['title'], 
            sample['content']
        )
        
        results.append({
            'post_id': sample['post_id'],
            'title': sample['title'],
            'gemini_selection': selection,
            'ground_truth_selection': sample['selected_indices'],
            'gemini_summary': summary,
            'ground_truth_summary': sample['golden']
        })
        
        # Delay để tránh rate limit
        if idx < num_samples - 1:
            time.sleep(delay)
    
    return pd.DataFrame(results)

# Chạy batch (uncomment để chạy)
# results_df = process_batch_with_gemini(df, num_samples=3, delay=2)
# results_df.to_csv(f"{output_dir}/gemini_results.csv", index=False, encoding='utf-8-sig')
# print("\n✅ Đã lưu kết quả vào gemini_results.csv")

print("💡 Uncomment dòng trên để chạy batch với Gemini API")

In [ ]:
# Hàm đánh giá kết quả (so sánh với ground truth)
import ast
import re

def parse_selection(selection_str):
    """Parse selection string thành list số"""
    try:
        # Tìm pattern [1, 2, 3, ...]
        match = re.search(r'\[[\d,\s]+\]', selection_str)
        if match:
            return ast.literal_eval(match.group())
        return []
    except:
        return []

def calculate_precision_recall_f1(predicted, ground_truth):
    """Tính Precision, Recall, F1"""
    predicted_set = set(predicted)
    ground_truth_set = set(ground_truth)
    
    if len(predicted_set) == 0:
        return 0, 0, 0
    
    tp = len(predicted_set & ground_truth_set)
    precision = tp / len(predicted_set) if len(predicted_set) > 0 else 0
    recall = tp / len(ground_truth_set) if len(ground_truth_set) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    
    return precision, recall, f1

# Ví dụ đánh giá
example_predicted = parse_selection("[1, 3, 5, 7, 9, 13]")
example_ground_truth = [0, 1, 6, 7, 13, 14, 15, 24, 25]

p, r, f1 = calculate_precision_recall_f1(example_predicted, example_ground_truth)
print("📊 Ví dụ đánh giá Selection:")
print(f"   Predicted: {example_predicted}")
print(f"   Ground Truth: {example_ground_truth}")
print(f"   Precision: {p:.2%}")
print(f"   Recall: {r:.2%}")
print(f"   F1 Score: {f1:.2%}")

## Thử nghiệm với Groq API

Groq API có tốc độ rất nhanh và hỗ trợ các model như Llama 3.3, Mixtral, Gemma.
- Lấy API key tại: https://console.groq.com/keys

In [ ]:
# Cài đặt thư viện Groq (chạy 1 lần)
# !pip install groq

from groq import Groq

# Cấu hình API Key - Thay YOUR_GROQ_API_KEY bằng API key của bạn
# Lấy API key tại: https://console.groq.com/keys
GROQ_API_KEY = "YOUR_GROQ_API_KEY"  # <-- Thay thế bằng API key của bạn

# Khởi tạo client
groq_client = Groq(api_key=GROQ_API_KEY)

# Các model có sẵn trên Groq (rất nhanh!)
# - llama-3.3-70b-versatile (mạnh nhất)
# - llama-3.1-8b-instant (nhanh nhất)
# - mixtral-8x7b-32768
# - gemma2-9b-it

GROQ_MODEL = "llama-3.3-70b-versatile"  # Có thể đổi sang model khác

print(f"Đã cấu hình Groq API với model: {GROQ_MODEL}")

In [ ]:
# Hàm gọi Groq API để chọn câu
def select_sentences_with_groq(title, content, num_sentences=6):
    """
    Sử dụng Groq để chọn các câu phù hợp với chủ đề
    """
    prompt = f"""Bạn là một chuyên gia phân tích văn bản tiếng Việt. 
Hãy chọn ra {num_sentences} câu phù hợp nhất với chủ đề sau: {title}

<list câu>
{content}
</list câu>

Trả về CHÍNH XÁC dưới dạng danh sách số, ví dụ: [1, 2, 3, 5, 8, 10]
Chỉ trả về danh sách số, không giải thích."""

    try:
        response = groq_client.chat.completions.create(
            model=GROQ_MODEL,
            messages=[
                {"role": "system", "content": "Bạn là một trợ lý phân tích văn bản tiếng Việt. Chỉ trả về kết quả, không giải thích."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.1,
            max_tokens=100
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Lỗi: {str(e)}"

# Hàm gọi Groq API để tóm tắt
def summarize_with_groq(title, content):
    """
    Sử dụng Groq để tóm tắt các câu bình luận
    """
    prompt = f"""Bạn là một chuyên gia tóm tắt văn bản tiếng Việt.
Hãy tóm tắt các bình luận dưới đây theo chủ đề: {title}

<list câu bình luận>
{content}
</list câu bình luận>

Yêu cầu:
- Tóm tắt ngắn gọn, súc tích (khoảng 3-5 câu)
- Giữ lại các ý kiến quan trọng
- Viết bằng tiếng Việt tự nhiên"""

    try:
        response = groq_client.chat.completions.create(
            model=GROQ_MODEL,
            messages=[
                {"role": "system", "content": "Bạn là một trợ lý tóm tắt văn bản tiếng Việt chuyên nghiệp."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3,
            max_tokens=500
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Lỗi: {str(e)}"

print("Đã định nghĩa các hàm gọi Groq API")

In [ ]:
# Thử nghiệm với Groq API - 1 mẫu dữ liệu
sample_idx = 0
sample = df.iloc[sample_idx]

print("=" * 60)
print("THỬ NGHIỆM VỚI GROQ API")
print("=" * 60)

print(f"\n📌 CHỦ ĐỀ: {sample['title']}")
print(f"\n📝 SỐ CÂU BÌNH LUẬN: {sample['num_comments']}")

# 1. Chọn câu với Groq
print("\n" + "-" * 60)
print("🔍 TASK 1: CHỌN CÂU PHÙ HỢP (GROQ)")
print("-" * 60)

groq_selection = select_sentences_with_groq(
    sample['title'], 
    sample['content_comment'],
    num_sentences=6
)

print(f"\n✅ Groq chọn: {groq_selection}")
print(f"📊 Ground Truth: {sample['selected_comment_indices']}")